<a href="https://colab.research.google.com/github/anilaksu/AI-and-Data-Science-Codes/blob/Natural-Language-Processing/Data_Science_Project_2_Password_Strength_with_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predict Password Strength using Natural Language Processing**


Anil Aksu

Personal e-mail: aaa293@cornell.edu

In a hutshell, we need to classify password data into three categories:


1.   0: Weak
2.   1: Medium
3.   2: Strong



## Notebook Organization:
- **Business Understanding**
- **Data Collection**
- **Data Cleaning**
- **Data Analysis**
- **Feature Engineering**
- **Model Building**
- **Deployment**

Full solution to the project can be found in the [link](https://drive.google.com/drive/folders/1ET_ggkzxtvQ287RyzHB5mcGxNyEbQe48)

In [1]:
# Here we set our working directory in our google drive to access datasets externally
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ColabNotebooks/Data Science Projects/Password Strength NLP Project

Mounted at /content/drive
/content/drive/MyDrive/ColabNotebooks/Data Science Projects/Password Strength NLP Project


#1. Data Collection

##1.1 Reading data from SQL Database